## HMM算法

In [1]:
import numpy as np

In [31]:
#0.初始化数据
Q = [1, 2, 3]
V = ['红', '白']

A = [[0.5, 0.2, 0.3],
    [0.3, 0.5, 0.2],
    [0.2, 0.3, 0.5]]

B = [[0.5, 0.5],
    [0.4, 0.6],
    [0.7, 0.3]]
O = ['红', '白', '红' ]
pi = [[0.2, 0.4, 0.4]]

In [32]:
class HiddenMarkov:
    def forward(self, Q, V, A, B, O, PI):
        """
        计算:p(o|lambda)
        """
        N = len(Q)  # 计算状态总数
        M = len(O)  # 计算观测总数
        
        alphas = np.zeros((N, M))  # 初始化空的alpha矩阵,每一个时刻都会有一个具体的观测变量
        T = M
        
        for t in range(T):  # 遍历所有时间节点来进行计算
            #  0.取得对应时刻观测序列的index
            indexOfO = V.index(O[t])
            #  1.初始化t=0时刻的alpha
            for i in range(N):
                if t == 0:
                    alphas[i][t] = PI[t][i] * B[i][indexOfO]  # i状态时刻所对应的观测量值
                
                    print("初始化状态的alpha: \n",alphas)
                else: 
                    alphas[i][t] = np.dot([alpha[t-1] for alpha in alphas],
                                         [a[i] for a in A] )* B[i][indexOfO]
                    
        
            
        p = np.sum([alpha[M-1] for alpha in alphas])  # 取得最后一个alpha序列
        print("Alphas矩阵:\n", alphas)
        print("A:\n", A)
        print("概率值为多少: ", p)
        
    def backward(self, Q, V, A, B, O, PI):
        """
        后项算法
        """
        N = len(Q)
        M = len(O)
        time = []
        
        betas = np.ones((N, M))
        
        for t in range(M-2, -1, -1):
            time.append(t)#记录时间序列
            indexOfO = V.index(O[t+1])#记录上一时刻的观测值
            
            for i in range(N):
                betas[i][t] = np.dot(
                np.multiply(A[i],[b[indexOfO] for b in B]),
                [beta[t+1] for beta in betas]
                )
                
        indexOfO = V.index(O[0])
        P = np.dot(np.multiply(PI, [b[indexOfO] for b in B]),
                  [beta[0] for beta in betas])#最后时刻的beta1
        print("观测序列值：",time)
        print("后项算法的概率值Ｐ：", P)
        print("Beta矩阵：\n", betas)
        
    def viterbi(self, Q, V, A, B, O, PI):
        """
        维特比算法：通过观测变量推测隐藏变量的值
        """
        #  0. 初始化数据
        N = len(Q)
        M = len(O)
        deltas = np.zeros((N, M))
        psis = np.zeros((N, M))
        I = np.zeros((1, M))
        
        for t in range(M):
            realT = t + 1
            indexOfO = V.index(O[t])  # 读取状态索引
            for i in range(N):
                realI = i + 1
        #  1. 初始化
                if t == 0:
                    deltas[i][t] = PI[0][i] * B[i][indexOfO]
                    psis[i][t] = 0
                
                else:
        #　２．递推
                    deltas[i][t] = np.max(
                    np.multiply([delta[t-1] for delta in deltas],
                               [a[i] for a in A])) * B[i][indexOfO]
                    
                    psis[i][t] = np.argmax(np.multiply([
                        delta[t - 1] for delta in deltas
                    ], [a[i] for a in A])) + 1 #返回索引值从零开始
                    
                    
                    
        #  3.终止条件
        I[0][M-1] = np.argmax([delta[M-1] for delta in deltas]) + 1
        
        #  4.选择路径．．
        for t in range(M-2, -1, -1):
            I[0][t] = psis[int(I[0][t+1]) -1][t + 1]#行数为所取得值（上一个状态的值, 列数为时间序列，＂Ｉ[0][t+1] - 1＂ 下标从零开始）
            print("行数：", (I[0][t+1]-1))
            print("列数：", (t+1))
            
        print("psis:", psis)    
        print("状态序列Ｉ:",I)  

### (1) 前向算法

In [33]:
HMM = HiddenMarkov()
HMM.forward(Q, V, A, B, O, pi)

初始化状态的alpha: 
 [[0.1 0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]]
初始化状态的alpha: 
 [[0.1  0.   0.  ]
 [0.16 0.   0.  ]
 [0.   0.   0.  ]]
初始化状态的alpha: 
 [[0.1  0.   0.  ]
 [0.16 0.   0.  ]
 [0.28 0.   0.  ]]
Alphas矩阵:
 [[0.1      0.077    0.04187 ]
 [0.16     0.1104   0.035512]
 [0.28     0.0606   0.052836]]
A:
 [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
概率值为多少:  0.130218


### (2)后向算法

In [34]:
HMM.backward(Q, V, A, B, O, pi)


观测序列值： [1, 0]
后项算法的概率值Ｐ： [0.130218]
Beta矩阵：
 [[0.2451 0.54   1.    ]
 [0.2622 0.49   1.    ]
 [0.2277 0.57   1.    ]]


### (3)维特比算法

In [35]:
HMM.viterbi(Q, V, A, B, O, pi)

行数： 2.0
列数： 2
行数： 2.0
列数： 1
psis: [[0. 3. 2.]
 [0. 3. 2.]
 [0. 3. 3.]]
状态序列Ｉ: [[3. 3. 3.]]


### 例题10.2

In [36]:
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['红', '白', '红', '红', '白', '红', '白', '白']
PI = [[0.2, 0.3, 0.5]]

In [37]:
HMM.forward(Q, V, A, B, O, PI)

初始化状态的alpha: 
 [[0.1 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]
初始化状态的alpha: 
 [[0.1  0.   0.   0.   0.   0.   0.   0.  ]
 [0.12 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]]
初始化状态的alpha: 
 [[0.1  0.   0.   0.   0.   0.   0.   0.  ]
 [0.12 0.   0.   0.   0.   0.   0.   0.  ]
 [0.35 0.   0.   0.   0.   0.   0.   0.  ]]
Alphas矩阵:
 [[0.1        0.078      0.04302    0.0218541  0.01146343 0.00576595
  0.0028617  0.00141148]
 [0.12       0.111      0.036684   0.0174942  0.01394714 0.00467612
  0.00338859 0.00169797]
 [0.35       0.0687     0.055965   0.03375771 0.00808018 0.00718798
  0.0018777  0.00074252]]
A:
 [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
概率值为多少:  0.0038519735794910995


In [38]:
HMM.backward(Q, V, A, B, O, PI)

观测序列值： [6, 5, 4, 3, 2, 1, 0]
后项算法的概率值Ｐ： [0.00385197]
Beta矩阵：
 [[0.00698005 0.01543889 0.02836522 0.05320483 0.116032   0.2149
  0.46       1.        ]
 [0.00741132 0.01370579 0.02649512 0.05776176 0.110117   0.2478
  0.51       1.        ]
 [0.00647031 0.01639589 0.02965701 0.04972891 0.122031   0.2023
  0.43       1.        ]]
